# Geopandas

Geopandas 可以用来快速处理 GIS 数据，它的操作逻辑和 pandas 相同。geopandas 中最主要的类型是 `geopandas.GeoSeries`，所有的操作都是基于 `geopandas.GeoSeries` 进行的。而 `geopandas.GeoDataFrame` 中需要指定一列作为 **geometry**，geometry 就是 `geopandas.GeoSeries`。

什么是 geometry？这个就需要去看一下 [`shapely`](https://shapely.readthedocs.io/en/stable/) 中的解释了。

## 读写数据 

### 从 `pandas.DataFrame` 中创建 `geopandas.GeoDataFrame`

通常来说，这是所有操作的起点，可以将 AIS 数据中的 latitude 和 longitude 转化为 geometry。

In [9]:
import geopandas as gpd
import pandas as pd

In [10]:
file_path = './data/ais_Dalian_201801'
df = pd.read_csv(f'{file_path}.csv', sep='|', header=0)

In [11]:
df.head()

,imo,timestamp,latitude,longitude,speed,heading
0,9244635,2018-01-31 23:17:54,39.010473,121.689498,0.0,359.0
1,9244635,2018-01-31 22:11:53,39.010498,121.689528,0.0,359.0
2,9244635,2018-01-31 21:05:51,39.010495,121.689470,0.0,359.0
3,9244635,2018-01-31 20:05:50,39.010525,121.689487,0.0,359.0
4,9244635,2018-01-31 18:53:48,39.010502,121.689485,0.0,359.0


In [12]:
gdf = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry=gpd.points_from_xy(df['longitude'], df['latitude']))
gdf.head()

,imo,timestamp,latitude,longitude,speed,heading,geometry
0,9244635,2018-01-31 23:17:54,39.010473,121.689498,0.0,359.0,POINT (121.68950 39.01047)
1,9244635,2018-01-31 22:11:53,39.010498,121.689528,0.0,359.0,POINT (121.68953 39.01050)
2,9244635,2018-01-31 21:05:51,39.010495,121.689470,0.0,359.0,POINT (121.68947 39.01049)
3,9244635,2018-01-31 20:05:50,39.010525,121.689487,0.0,359.0,POINT (121.68949 39.01053)
4,9244635,2018-01-31 18:53:48,39.010502,121.689485,0.0,359.0,POINT (121.68949 39.01050)


### 将 `geopandas.GeoDataFrame` 保存为 `.geojson` 文件

`.geojson` 是保存 GIS 数据的一种常用文件类型。

In [13]:
gdf.to_file(f'{file_path}.geojson', driver='GeoJSON')

### 从 `.geojson` 中创建 `geopandas.GeoDataFrame`

In [14]:
gdf = gpd.read_file(f'{file_path}.geojson')
gdf.head()

,imo,timestamp,latitude,longitude,speed,heading,geometry
0,9244635,2018-01-31T23:17:54,39.010473,121.689498,0.0,359.0,POINT (121.68950 39.01047)
1,9244635,2018-01-31T22:11:53,39.010498,121.689528,0.0,359.0,POINT (121.68953 39.01050)
2,9244635,2018-01-31T21:05:51,39.010495,121.689470,0.0,359.0,POINT (121.68947 39.01049)
3,9244635,2018-01-31T20:05:50,39.010525,121.689487,0.0,359.0,POINT (121.68949 39.01053)
4,9244635,2018-01-31T18:53:48,39.010502,121.689485,0.0,359.0,POINT (121.68949 39.01050)


## Coordinate Reference Systems

Coordinate Reference Systems（CRS）是 GIS 中一个非常重要的概念。在我们记录的数据中，通常只保留有点的坐标信息。我们必须告诉程序如何将坐标投影到地球上的真实点。

AIS 数据中记录的是经纬度坐标，它的 CRS 是 WGS84 latitude-longitude projection，是世界上最常见的 CRS，也称作 EPSG:4326，它的单位（unit）是度（degree）。

有时候我们需要以米（meter）为单位，这就需要我们更换成以米为单位的 CRS，需要注意的是，将地球转化为平面的过程中，难免会出现拉伸、变形，导致准确度下降。因此，不同的 CRS 是有使用范围（area of use）的。另一个较为常用的以米为单位的 CRS 是 EPSG:3857，它的使用范围是 $85^\circ S \sim 85^\circ N$，在低纬度时，精度较高，在高纬度时，变形程度较大。考虑到港口、航线在高纬地区较少，因此我认为，在 AIS 相关计算中使用 EPSG:3857 同样是比较合理的。

| CRS       | Unit   | Area of use                                     |
| --------- | ------ | ----------------------------------------------- |
| EPSG:4326 | degree | World                                           |
| EPSG:3857 | meter  | World between $85^\circ.06S$ and $85^\circ.06N$ |

利用 geopandas，我们可以进行 CRS 的快速变换。

### 查看 geometry 的 CRS

In [16]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### 转换 geometry 的 CRS

In [17]:
gdf = gdf.to_crs('EPSG:3857')
gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich